In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns

# Bootstrap Tutorial

This notebook contains a tutorial on how to use the bootstrap functionality provided by estimagic. In this example, we will work with the "exercise" example dataset taken from the seaborn library.

The working example will be a linear regression to investigate the effects of exercise time on pulse.

In [2]:
df = sns.load_dataset("exercise")

dictionary = {"1 min": 1, "15 min": 15, "30 min": 30}

df = df.replace({"time": dictionary})
df["constant"] = 1

print(df.head())

   Unnamed: 0  id     diet  pulse  time  kind  constant
0           0   1  low fat     85     1  rest         1
1           1   1  low fat     85    15  rest         1
2           2   1  low fat     88    30  rest         1
3           3   2  low fat     90     1  rest         1
4           4   2  low fat     92    15  rest         1


In general, there are two ways to use bootstrap functionality in estimagic. We can either use the main bootstrap() function, which will draw bootstrap samples, compute estimates, as well as compute standard errors and confidence intervals of these estimates at the same time. Or we can use the fact that all of the code is separated, and go through all of this step by step. 

In either way, the first thing that is required from the user is to specify a wrapping function that calculates the statistic of interest, and only takes the data on which to compute the statistic as an input. In our case, we want to regress "pulse" on "time" and a constant, and the function looks as follows:

In [3]:
def ols_fit(data):
    
    y = data["pulse"]
    x = data[["constant", "time"]]
    
    params = pd.Series(
        sm.OLS(y, x).fit().params, index=["constant", "time"]
    )
    
    return params

By returning a pd.Series with the coefficient names as index, we make sure we will eventually get a nice and readable output table. This is however not necessary. We could also simply return the parameter vector, in which case we would simply have to keep in mind the meaning of each parameter by ourselves.

## The Easy Way

After defining our function of interest, we can make use of estimagic's bootstrap functionality using one simple function call of the bootstrap() function.

In [4]:
from estimagic.inference.bootstrap import bootstrap

results_without_cluster = bootstrap(data=df, f=ols_fit)

print(results_without_cluster)

               mean       std   lower_ci   upper_ci
constant  93.747665  1.505694  90.748863  96.774676
time       0.392136  0.124565   0.154251   0.645340


The above function call represents the minimum that a user has to specify, making full use of the default options, such as drawing a 1000 bootstrap draws, using the "percentile" bootstrap confidence interval, not making use of parallelization, etc.

If we would for example want to make 10000 draws, while parallelizing the maximum possible number of threads and using a "bca" type confidence interval, we would simply call the following.

In [5]:
results_without_cluster2 = bootstrap(data=df, f=ols_fit, ndraws=10000, ci_method="bca", n_cores=-1)

print(results_without_cluster2)

               mean       std   lower_ci   upper_ci
constant  93.784457  1.512215  91.813764  97.294586
time       0.386832  0.126028   0.126121   0.563412


Estimagic implements two different types of bootstrap. In the standard bootstrap that we called in the two examples above, observations from the original dataset are drawn uniformly with replacement in order to create the different bootstrap samples.

In the cluster robust variant of the bootstrap, the original dataset is divided into clusters according to the values of some user-specified variable, and then clusters are drawn uniformly with replacement in order to create the different bootstrap samples. 

In order to use the cluster robust boostrap, we simply specify which variable to cluster by. In the example we are working with, it seems sensible to cluster on individuals, i.e. on the column "id" of our dataset.

In [6]:
results_with_cluster = bootstrap(data=df, f=ols_fit, cluster_by="id")

print(results_with_cluster)

               mean       std   lower_ci   upper_ci
constant  93.783383  1.157161  91.469985  95.996858
time       0.387834  0.101596   0.206891   0.597352


We can see that the estimated standard errors are indeed of a smaller magnitude when we use the cluster robust bootstrap.

Finally, we can compare the results to full sample regressions using the statsmodels regression function and see that the cluster robust bootstrap gives standard error estimates very close to the cluster robust regression, while the regular bootstrap seems to overestimate the standard errors of both coefficients.

In [7]:
y = df["pulse"]
x = df[["constant", "time"]]


cluster_robust_ols = sm.OLS(y, x).fit(
        cov_type="cluster", cov_kwds={"groups": df["id"]}
    )

cluster_robust_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  pulse   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     13.75
Date:                Sat, 07 Mar 2020   Prob (F-statistic):           0.000879
Time:                        10:23:19   Log-Likelihood:                -365.51
No. Observations:                  90   AIC:                             735.0
Df Residuals:                      88   BIC:                             740.0
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
constant      93.7611      1.205     77.837      0.000      91.400      96.122
time           0.3873      0.104      3.708      0.000       0.183       0.592
==============================================================================
Omnibus:                       20.828   Durbin-Watson:                   0.827
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               26.313
Skew:                           1.173   Prob(JB):                     1.93e-06
Kurtosis:                       4.231   Cond. No.                         31.7
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

Finally, note that we can also call the bootstrap function (and all functions below) with a list of statistics instead of one single statistic function.

In [8]:
def corr(data):
    
    return pd.Series(data["pulse"].corr(data["time"]), index=["corr(pulse, time)"])

list_results = bootstrap(data=df, f=[ols_fit, corr])

print(list_results)

                        mean       std   lower_ci   upper_ci
constant           93.808116  1.468092  90.866438  96.592118
time                0.389905  0.124758   0.160019   0.641667
corr(pulse, time)   0.307628  0.080118   0.150206   0.453532


## The More Sophisticated Way

In some analyses, we might be interested in not simply calculating one statistic of our sample, but indeed calculating several statistics of our sample one after another. Here I want to show how this can be done with bootstrap in estimagic.

In a first step, we should fix seeds in order to make sure that the samples we base our estimates on are the same for each separate statistic. The seeds returned are simply an np.array of ndraws valid random seeds.

In [9]:
from estimagic.inference.bootstrap_samples import get_seeds

my_seeds = get_seeds(2000)

print(my_seeds)

[ 820832367  418597563 1743692288 ... 1101394466 1215746329 1456628164]


Next, we can then go on to call the bootstrap function for different statistics, e.g. the above defined function and also a simple correlation coefficient of "pulse" and "time":

In [10]:
results_ols = bootstrap(data=df, f=ols_fit, seeds=my_seeds)
print(results_ols)

               mean       std   lower_ci   upper_ci
constant  93.840476  1.525184  90.884772  96.924009
time       0.383560  0.127032   0.150264   0.647411


In [11]:
results_corr = bootstrap(data=df, f=corr, seeds=my_seeds)
print(results_corr)

                       mean       std  lower_ci  upper_ci
corr(pulse, time)  0.304974  0.082786  0.140951  0.459555


## A Complete Separation of Code

There might of course also be situations where we simply want to return the collection of bootstrap estimates and not directly get the very streamlined results table. The following example makes use of the complete separation of code.

In [12]:
more_seeds = get_seeds(2000)

Next, we can use get_bootstrap_estimates() to get the pd.DataFrame containing all of the different bootstrap estimates for f. We could again use cluster_by to use the clustered bootstrap.

In [13]:
from estimagic.inference.bootstrap_estimates import get_bootstrap_estimates

my_estimates = get_bootstrap_estimates(data=df, f=ols_fit, seeds=more_seeds)

print(my_estimates.head())

    constant      time
0  94.566718  0.210447
1  93.308101  0.385386
2  92.484300  0.450313
3  92.987570  0.478580
4  95.768208  0.271246


We can also compute confidence intervals using estimates as an input.

In [14]:
from estimagic.inference.bootstrap_ci import compute_ci

confidence_intervals = compute_ci(data=df, f=ols_fit, estimates=my_estimates, ci_method="bc")

print(confidence_intervals)

           lower_ci   upper_ci
constant  91.552669  96.522910
time       0.181187   0.593919


Alternatively, we can compute the whole results table using specific estimates as an input.

In [15]:
from estimagic.inference.bootstrap import get_results_table

my_results = get_results_table(data=df, f=ols_fit, estimates=my_estimates, ci_method="t")

print(my_results)

               mean       std   lower_ci   upper_ci
constant  93.742363  1.506603  90.691330  96.584602
time       0.393578  0.127448   0.124622   0.613314


Finally, note that it is also possible to return the list of bootstrap samples using the get_bootstrap_samples() function. This function returns a list of ndraws np.arrays of bootstrap sample ids (row numbers) obtained from drawing from the original sample with replacement, using the cluster robust bootstrap if specified using the cluster_by input.
If we specify return_samples=True, the function instead returns a list of DataFrames. This can however be very memory inefficient for large datasets and/or large numbers of bootstrap draws.

In [16]:
from estimagic.inference.bootstrap_samples import get_bootstrap_samples

my_samples = get_bootstrap_samples(data=df, seeds=my_seeds)

print(type(my_samples))

print(my_samples[4])

<class 'list'>
[43 69 40  9 61 51 68 84 51 89 78 11 68 47 64 59 47  2 19 82 76 17 65 35
 73 59 60 18 40 66 77 24  6 10 14 88 20 21  1 48 41 30 79 61 55  2 83  1
 82 47 57 55 54 81 43 15 43  2 30 75 26 67 89 50 56 28  0  0 50 54 40 54
 66 47 84 69 56 79 39 39 38 54  7 55 82 40 83 31 38 25]
